# Projeto 1 - módulo 6

## Precificação dinâmica - e-commerce

### Mercari Price Suggestion Challenge - Kaggle

Mercari é um site de revenda de produtos online. Uma dos desafios desse tipo de plataforma é auxiliar o usuário, muitas vezes com pouco conhecimento de vendas, a determinar um preço para os seus produtos de modo a maximizar as chances de venda.

### Sobre este projeto

O presente projeto tem o objetivo de desenvolver um algoritmo que identifique produtos já vendidos similares e sugira ao usuário um preço ótimo para novos produtos cadastrados.


### Preparação do ambiente

Para este projeto, acesse o link https://www.kaggle.com/competitions/mercari-price-suggestion-challenge/overview 


In [ ]:
!pip install nlp
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nlp
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import r2_score , mean_absolute_error as MAE, mean_squared_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_train = pd.read_csv ('/content/drive/MyDrive/Blue Edtech/data/processed/train_data.csv') 
base_test = pd.read_csv ('/content/drive/MyDrive/Blue Edtech/data/processed/test_data.csv') 

In [ ]:
base_train

,name,category_1,category_2,category_3,item_condition_id,brand_name,price,shipping,item_description,date,stock
0,iphone 6s plus defender case,Electronics,Cell Phones & Accessories,"Cases, Covers & Skins",1,No Brand,10.0,1,protective case for the big iphones 6plus or 6...,15-6-2018,2
1,beautiful rainbow eye iphone plus case,Electronics,Cell Phones & Accessories,"Cases, Covers & Skins",1,No Brand,7.0,1,this soft case is brand new and features a bea...,12-12-2018,1
2,hold under armour women s t shirt bundle,Women,Athletic Apparel,Shirts & Tops,2,Under Armour,20.0,1,women s under armour t shirts with freedom log...,13-5-2018,11
3,nike shorts bundle,Women,Athletic Apparel,Shorts,3,Nike,18.0,1,no description yet,24-2-2018,4
4,bare mineral eye shadow lot,Beauty,Makeup,Eyes,3,No Brand,14.0,0,bare mineral eye shadow verity of colors bare ...,29-10-2018,11
...,...,...,...,...,...,...,...,...,...,...,...
1037157,herschel backpack,Women,Women's Handbags,Backpack Style,2,No Brand,20.0,0,herschel packable daypack only used a couple o...,30-7-2018,14
1037158,illamasqua beyond powder in omg,Beauty,Makeup,Face,1,No Brand,25.0,1,product details a baked highlighting powder su...,19-8-2018,7
1037159,free ship tracking,Other,Office supplies,Shipping Supplies,1,No Brand,3.0,1,for,29-12-2018,7
1037160,pink 3ds xl,Electronics,Video Games & Consoles,Consoles,3,Nintendo,85.0,1,comes with charger the stylus is not original,23-3-2018,7


In [ ]:
base_train['item_condition_id'] = base_train['item_condition_id'].map({1: 'otimo', 2: 'bom', 3: 'regular', 4: 'ruim', 5: 'pessimo'}, na_action = None)
base_test['item_condition_id'] = base_test['item_condition_id'].map({1: 'otimo', 2: 'bom', 3: 'regular', 4: 'ruim', 5: 'pessimo'}, na_action = None)

In [ ]:
base_train['texto'] = base_train['name'] + ' ' + base_train['category_1'] + ' ' + base_train['category_2'] + ' ' + base_train['category_3'] + ' ' + base_train['brand_name'] + ' ' + base_train['item_condition_id'] 
base_test['texto'] = base_test['name'] + ' ' + base_test['category_1'] + ' ' + base_test['category_2'] + ' ' + base_test['category_3'] + ' ' + base_test['brand_name'] + ' ' + base_test['item_condition_id']

In [ ]:
import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# extraindo características

textos_train = base_train[['texto']]
stop_words = stopwords.words('english')
tf = TfidfVectorizer(analyzer = 'word', min_df=0, stop_words = stop_words)
tfidf_matrix_train = tf.fit_transform(textos_train['texto'].values.astype('U'))
textos_test = base_test[['texto']]
tfidf_matrix_test = tf.transform(textos_test['texto'].values.astype('U'))
print(tfidf_matrix_train.shape)
tfidf_matrix_test.shape

(1037162, 85247)


(444499, 85247)

In [ ]:
def precificacao_cosseno(produto):

  # calculando a similaridade de coseno do produto selecionado com os demais

  medida = cosine_similarity(tfidf_matrix_test[produto], tfidf_matrix_train)
  select = (base_test.index == produto)
  produto_cadastrado = base_test[select]

# identificando os produtos com coeficiente de similaridade maior ou igual a 0.5

  pos_maximo = np.where(medida[0] >= 0.5 )
  indicacao = base_train.iloc[pos_maximo].price.mean()
  return indicacao

In [ ]:
# # Teste você mesmo!

# produto = int(input('Digite índice do produto: \n'))
# select = (base_test.index == produto)
# produto_cadastrado = base_test[select]
# produto_cadastrado[['name', 'category_1', 'category_2', 'category_3', 'item_description', 'item_condition_id', 'brand_name', 'price']]

In [ ]:
# # lista de produtos similares

# resultado = precificacao_cosseno(produto)
# resultado[['name', 'category_1', 'category_2', 'category_3', 'item_description', 'item_condition_id', 'brand_name', 'price']]

In [ ]:
# preco_min = resultado['price'].min()
# preco_max = resultado['price'].max()
# preco_medio = resultado['price'].mean()
# preco_moda = resultado['price'].mode()
# print(preco_min, preco_max, preco_medio, preco_moda)
# print(produto_cadastrado['price'])

In [ ]:
preco_medio = pd.DataFrame(columns = ['price'])

for produto in range(0,10000):
  resultado = precificacao_cosseno(produto)
  if resultado > 0:
    preco_medio.loc[produto] = [resultado]
  else:
    preco_medio.loc[produto] = [0]

In [ ]:
# adaptando o tamanho do teste para calcular as métricas

y_test = base_test.iloc[:10000,:]

In [ ]:
# calculando as métricas do modelo
print('Root Mean squared erro: %.2f' % np.sqrt(mean_squared_error(y_test['price'], preco_medio['price'])))
print('erro médio absoluto: %.2f' % MAE(y_test['price'], preco_medio['price']))
print('R score: %.2f' % r2_score(y_test['price'], preco_medio['price']))

In [ ]:
preco_medio['price real'] = y_test['price']

In [ ]:
preco_medio